## 03 - Forecasting

### Import packages and load the data

In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import math
import matplotlib.pyplot as plt
import numpy as np
import datetime
# from flaml import AutoML
import plotly.express as px
import pickle

In [2]:
data_input_path = '/Users/szejozsef00/Desktop/MSC/MSC 2. félév/DS Lab I/DSLAB1/data/processed/'
prediction_output_path = '/Users/szejozsef00/Desktop/MSC/MSC 2. félév/DS Lab I/DSLAB1/data/predictions/'

data = pd.read_csv(data_input_path + 'modelling_df.csv',parse_dates=['DATETIME'])
data.head(5)

,DATE,DATETIME,LOCATION,VALUE,VALUE_SCALED,All Saints' Day,Christmas Day,Easter Monday,Labor Day,New Year's Day,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,2009-07-02,2009-07-02 00:00:00,0,-79.50,0.425602,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2009-07-02,2009-07-02 00:05:00,0,-22.81,0.463518,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2009-07-02,2009-07-02 00:10:00,0,23.02,0.494171,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2009-07-02,2009-07-02 00:15:00,0,21.36,0.493061,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2009-07-02,2009-07-02 00:20:00,0,25.18,0.495616,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Create features

In [3]:
# Creating the features used for prediction
def create_features(data, n_lags=7, feature_name=None, horizont=7):
    features = data.sort_values(['LOCATION', 'DATETIME'])
    features.drop(columns=[feature_name], inplace=True)

    # Create lag features

    for lag in range(horizont, 7*horizont+1, horizont):
        features[f'{feature_name}_lag{lag}'] = data[['LOCATION', f'{feature_name}']].groupby('LOCATION').shift(lag)

    
    features[feature_name] = data[f'{feature_name}']
    feature_list = features.columns.tolist()

    return features, feature_list

In [4]:
# Defining parameters for the prediction
# The target variable we want to predict
target = 'VALUE_SCALED'

# The start date for the predictions
prediction_start_date = '2010-07-01'

# Confidence level for the lower/upper bound
alpha = 0.7

# Horizont of the predictions
horizont = 288*3

# Preparing data
data.reset_index(drop=True, inplace=True)
data.loc[data['DATE'] >= prediction_start_date, target] = None


# Creating features
data_all, feature_list = create_features(data, 
                                         2*horizont, 
                                         feature_name=target, 
                                         horizont=horizont)

# LightGBM parameters
params = {
       'objective': 'regression', 
       'metric': 'mape',
       'boosting_type': 'gbdt',  # Gradient boosting decision tree
       'learning_rate': 0.05,
       'num_leaves': 100, #31
       'feature_fraction': 0.9,
       'bagging_fraction': 0.8,
       'bagging_freq': 5
}

# Alternative parameters for quantile regression
lower_params = {
    'objective': 'quantile',  
    'alpha': 1 - alpha, 
    'boosting_type': 'gbdt', 
    'learning_rate': 0.05,
    'num_leaves': 100,  # 31
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
}

upper_params = {
    'objective': 'quantile',  
    'alpha': alpha, 
    'boosting_type': 'gbdt', 
    'learning_rate': 0.05,
    'num_leaves': 100,  # 31
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
}

In [5]:
# Finding the start and end of the prediction period
prediction_start_date = pd.to_datetime(prediction_start_date)
prediction_end_date = pd.to_datetime(prediction_start_date) + datetime.timedelta(days=7) - datetime.timedelta(minutes=5)

In [6]:
# Creating the list of dates for prediction
dates = pd.date_range(start=prediction_start_date, end=prediction_end_date, freq='5T')
dates = dates.append(pd.DatetimeIndex([prediction_end_date]))

In [8]:
def inverse_transform_by_location(df_scaled, scalers_path, value_col='predicted', result_col = 'predicted_scaled', location_col='LOCATION'):
    # Load the scalers from the pickle file
    with open(scalers_path, 'rb') as f:
        scalers = pickle.load(f)
    
    # Create a copy of the dataframe to avoid modifying the original one
    df_original = df_scaled.copy()
    
    # Iterate over each unique location
    for location in df_scaled[location_col].unique():
        # Filter the dataframe for the current location
        location_df = df_scaled[df_scaled[location_col] == location]
        
        # Get the scaler for the current location
        scaler = scalers[location]
        
        # Inverse transform the VALUE_SCALED column for the current location
        df_original.loc[df_scaled[location_col] == location, result_col] = scaler.inverse_transform(location_df[[value_col]])
    
    return df_original

In [9]:
# Defining the function for the prediction
def predict_future(data, prediction_start_date, prediction_end_date, target, horizont):

    # Separating past and future data
    dt_past = data[data['DATETIME'] < prediction_start_date].dropna()
    dt_future = data[(data['DATETIME'] >= prediction_start_date) & (data['DATETIME'] <= prediction_end_date)]      

    # Separating predictors and target 
    X = dt_past.drop(columns=['DATE','LOCATION', 'DATETIME', target])
    y = dt_past[target]  

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Create LightGBM dataset
    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

    # Train the main regression model
    model = lgb.train(
        params,
        train_data,
        valid_sets=[train_data, test_data],
        num_boost_round=100,
        callbacks=[lgb.early_stopping(stopping_rounds=100)]
    )

    # Train lower quantile model
    lower = lgb.train(
        lower_params,
        train_data,
        valid_sets=[train_data, test_data],
        num_boost_round=100,
        callbacks=[lgb.early_stopping(stopping_rounds=100)]
    )

    # Train upper quantile model
    upper = lgb.train(
        upper_params,
        train_data,
        valid_sets=[train_data, test_data],
        num_boost_round=100,
        callbacks=[lgb.early_stopping(stopping_rounds=100)]
    )

    # Calculate contributions and predictions
    contributions = model.predict(
        dt_future.drop(columns=['DATE','LOCATION', 'DATETIME', target]),
        num_iteration=model.best_iteration,
        pred_contrib=True
    )

    dt_future['predicted_scaled'] = model.predict(
        dt_future.drop(columns=['DATE','LOCATION', 'DATETIME', target]),
        num_iteration=model.best_iteration, predict_disable_shape_check=True
    )
    dt_future['lower_scaled'] = lower.predict(
        dt_future.drop(columns=['DATE','LOCATION', 'DATETIME', target]),
        num_iteration=model.best_iteration, predict_disable_shape_check=True
    )
    dt_future['upper_scaled'] = upper.predict(
        dt_future.drop(columns=['DATE','LOCATION', 'DATETIME', target]),
        num_iteration=model.best_iteration, predict_disable_shape_check=True
    )

    # Example usage
    scalers_path = data_input_path + 'scalers.pkl'
    dt_future = inverse_transform_by_location(dt_future, scalers_path,value_col='predicted_scaled',result_col='prediction', location_col='LOCATION')
    dt_future = inverse_transform_by_location(dt_future, scalers_path,value_col='lower_scaled',result_col='lower_bound', location_col='LOCATION')
    dt_future = inverse_transform_by_location(dt_future, scalers_path,value_col='upper_scaled',result_col='upper_bound', location_col='LOCATION')
  
    # Add the prediction horizon
    dt_future['horizont'] = horizont

    return dt_future[['LOCATION', 'DATETIME', 'horizont', 'prediction', 'lower_bound', 'upper_bound']]

In [10]:
# Get the feature list
features_wo_lag = [f for f in feature_list if "lag" not in f]
features_w_lag = [f for f in feature_list if "lag" in f]
features = features_wo_lag + features_w_lag
# features = features_w_lag + ['DATETIME','DATE','LOCATION','VALUE_SCALED']

In [11]:

df = data_all[features]

In [12]:
# Prediction
prediction = predict_future(df, prediction_start_date, prediction_end_date, target, horizont=288)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.137095 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2162
[LightGBM] [Info] Number of data points in the train set: 15568358, number of used features: 69
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 0.452111
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	training's mape: 0.0566686	valid_1's mape: 0.0464363
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.182901 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2162
[L

/var/folders/80/jvk9kxf57t13l590hz_z_jcr0000gn/T/ipykernel_3582/689576697.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt_future['predicted_scaled'] = model.predict(
/var/folders/80/jvk9kxf57t13l590hz_z_jcr0000gn/T/ipykernel_3582/689576697.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt_future['lower_scaled'] = lower.predict(
/var/folders/80/jvk9kxf57t13l590hz_z_jcr0000gn/T/ipykernel_3582/689576697.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [13]:
# Calculate MAPE fpr all predictions
dt_plot = data_all[['DATETIME', 'LOCATION', 'VALUE']].merge(prediction, how='left', on = ['LOCATION', 'DATETIME'])
a = dt_plot[(~dt_plot.prediction.isna())].sort_values(by=['LOCATION','DATETIME'])

In [14]:
# Calculate MAPE by location
mape_by_location = a.groupby('LOCATION').apply(lambda x: mean_absolute_percentage_error(x['VALUE'], x['prediction'])).reset_index(name='MAPE')
rmse_by_location = a.groupby('LOCATION').apply(lambda x: math.sqrt(mean_squared_error(x['VALUE'], x['prediction']))).reset_index(name='RMSE')
stat_df = pd.concat([mape_by_location, rmse_by_location['RMSE']], axis=1)

In [15]:
print("MAPE" ,mean_absolute_percentage_error(a['VALUE'], a['prediction']))
print("RMSE" ,math.sqrt(mean_squared_error(a['VALUE'], a['prediction'])))

MAPE 7.800556305731947e+16
RMSE 343.1452197241382


In [34]:
stat_df.sort_values(by='MAPE', ascending=True).head(20)

,LOCATION,MAPE,RMSE
110,110,0.020728,34.537074
108,108,0.021388,36.005269
16,16,0.022834,47.980814
19,19,0.044755,60.778480
143,143,0.051612,82.836860
17,17,0.089087,91.931706
192,192,0.091862,85.756657
172,172,0.094663,51.428373
115,115,0.101601,72.851351
28,28,0.103740,53.460225


In [31]:
def plot_location(location_id):
    location_data = a[a['LOCATION'] == location_id]
    mape_value = stat_df[stat_df['LOCATION'] == location_id]['MAPE'].values[0]
    rmse_value = stat_df[stat_df['LOCATION'] == location_id]['RMSE'].values[0]
    fig = px.line(location_data, x='DATETIME', y=['VALUE', 'prediction'], labels={
        'DATETIME': 'Datetime',
        'value': 'Value',
        'prediction': 'Prediction'
    }, title=f'Prediction vs Actual Values for Location {location_id} (MAPE: {mape_value:.5f}, RMSE: {rmse_value:.5f})')
    fig.show()

for loc in range(3,7,1):
    plot_location(loc)

In [17]:
prediction_output_path = '/Users/szejozsef00/Desktop/MSC/MSC 2. félév/DS Lab I/DSLAB1/data/predictions/'

In [18]:
prediction.to_csv(prediction_output_path + 'pred_2010_07_01_03_0.7.csv', index=False)